# Breast Cancer Classification

# Import packages

In [1]:
#!pip install scikit-learn opencv-python 

In [2]:
#pip install torch==2.0+cu117 --user -f https://download.pytorch.org/whl/cu117/torch_stable.html

In [3]:
#!pip install torchvision==0.15.0

In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import glob
#import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
import random
import cv2
import sys
from torchvision import models,transforms,datasets
import time
import os
import torch.nn.functional as F
from torchvision.datasets.folder import default_loader
import torchvision
from sklearn .metrics import roc_auc_score,accuracy_score,precision_score,recall_score,f1_score
from PIL import Image
from torch.autograd import Variable
import torch.optim.lr_scheduler as lr_scheduler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

In [5]:
print(torch.cuda.is_available()) # this should return true
torch.cuda.get_device_name() # this should return your graphics card name. Ex) 'NVIDIA RTX A4000'

True


'NVIDIA RTX A4000'

In [6]:
# # importing the zipfile module 


# from zipfile import ZipFile 

#  # loading the temp.zip and creating a zip object 
# with ZipFile("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis_ddsm.zip", 'r') as zObject: 
#     zObject.extractall() 



In [7]:
import pandas as pd
dicom_data = pd.read_csv("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\dicom_info.csv")

full_mammogram_images = dicom_data[dicom_data.SeriesDescription == 'full mammogram images'].image_path
full_mammogram_images.head()

1     CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.248386...
2     CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.267213...
11    CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.210396...
12    CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.749566...
15    CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.987658...
Name: image_path, dtype: object

In [ ]:
# load the mass dataset
mass_train = pd.read_csv('C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\mass_case_description_train_set.csv')
mass_test = pd.read_csv('C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\mass_case_description_test_set.csv')

#mass_train.head()
mass_train.iloc[0][11].split("/")[2]

# fix image paths
def fix_image_path(data):
 #   correct dicom paths to correct image paths
  for index, img in enumerate(data.values):
    img_name = img[11].split("/")[2]
    new_path = "C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\jpeg\\" + img_name
    print(new_path)
    data.iloc[index,11] = new_path
      
# apply to datasets
fix_image_path(mass_train)
fix_image_path(mass_test)


In [ ]:

calc_train = pd.read_csv('C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\calc_case_description_train_set.csv')
calc_test = pd.read_csv('C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\calc_case_description_test_set.csv')

fix_image_path(calc_train)
fix_image_path(calc_test)


In [10]:
temp_calc_train = calc_train[["image file path","pathology","abnormality id"]]
calc_normal = []
calc_malignant = []
for row,col in temp_calc_train.iterrows():
    if col["pathology"] == "MALIGNANT":
        calc_malignant.append(col["image file path"])
    else:
        calc_normal.append(col["image file path"])

print(len(calc_normal))
print(len(calc_malignant))
print(len(temp_calc_train))


1002
544
1546


In [11]:
new_df = mass_train[["image file path","pathology","abnormality id"]]

normal_path = []
malignant_path = []
for row,col in new_df.iterrows():
    if col["pathology"] == "MALIGNANT":
        # put this file to the malignant folder
        malignant_path.append(col["image file path"])
    else:
        normal_path.append(col["image file path"])

print(len(normal_path))
print(len(malignant_path))
print(len(new_df))


681
637
1318


In [12]:
"""
path = "C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\"
print(len(os.listdir(os.path.join(path,"cbis-test\\malignant"))))
print(len(os.listdir(os.path.join(path,"cbis-test\\normal"))))
print(len(os.listdir(os.path.join(path,"cbis-train\\malignant"))))
print(len(os.listdir(os.path.join(path,"cbis-train\\normal"))))
print(len(os.listdir(os.path.join(path,"cbis-val\\malignant"))))
print(len(os.listdir(os.path.join(path,"cbis-val\\normal"))))
"""

'\npath = "C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\"\nprint(len(os.listdir(os.path.join(path,"cbis-test\\malignant"))))\nprint(len(os.listdir(os.path.join(path,"cbis-test\\normal"))))\nprint(len(os.listdir(os.path.join(path,"cbis-train\\malignant"))))\nprint(len(os.listdir(os.path.join(path,"cbis-train\\normal"))))\nprint(len(os.listdir(os.path.join(path,"cbis-val\\malignant"))))\nprint(len(os.listdir(os.path.join(path,"cbis-val\\normal"))))\n'

In [13]:
last_df = pd.concat([temp_calc_train,new_df])
calc_test = calc_test[["image file path","pathology","abnormality id"]]
mass_test = mass_test[["image file path","pathology","abnormality id"]]
new_test = pd.concat([calc_test,mass_test])

#last_df.drop_duplicates(subset=["image file path"],inplace = True)
#new_test.drop_duplicates(subset=["image file path"],inplace=True)

train = last_df[:2160]
val = last_df[2160:]
test_df = new_test

C:\Users\ae423\OneDrive - University of Sussex\Desktop\AlphanElmasDissertation\jpeg\1.3.6.1.4.1.9590.100.1.2.47414316010368386519740343172775938548

In [14]:
#os.listdir("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\jpeg\\1.3.6.1.4.1.9590.100.1.2.47414316010368386519740343172775938548")

In [15]:
# import shutil
# p1 = 'C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\jpeg\\1.3.6.1.4.1.9590.100.1.2.406725628213826290127343763811145520834\\1-192.jpg'
# p2 = "C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\a\\b"
# p3 = "C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\a\\c"
# shutil.copy(p1,p2)
# shutil.copy(p1,p3)
# # ayni file li ayni foldera iki kere kaydetmiyor

In [16]:
# res= []
# import shutil

# def write_into_train_folder():
#     for row,col in train.iterrows():
        
#         for img_name in os.listdir(col["image file path"]):
#             img_path = col["image file path"] + "\\" + img_name
#             if col["pathology"] == "MALIGNANT":
#                 # put this file to the malignant folder
#                 new_path = "C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-train\\malignant\\" + str(col["abnormality id"]) + "-" + col["image file path"].split("\\")[-1] + "-" + img_name
#                 #res.append(new_path)
#                 shutil.copy(img_path,new_path)
#             else:
#                 new_path = "C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-train\\normal\\" + str(col["abnormality id"]) + "-" + col["image file path"].split("\\")[-1] + "-" + img_name
#                 #res.append(new_path)
#                 shutil.copy(img_path,new_path)
#                 # put it into normal folder


# def write_into_val_folder():
#     for row,col in val.iterrows():
#         #res.append(os.listdir(col["image file path"]))
#         for img_name in os.listdir(col["image file path"]):
#             img_path = col["image file path"] + "\\" + img_name
#             if col["pathology"] == "MALIGNANT":
#                 # put this file to the malignant folder
#                 new_path = "C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-val\\malignant\\" + str(col["abnormality id"]) + "-" + col["image file path"].split("\\")[-1] + "-" + img_name
#                 #res.append(new_path)
#                 shutil.copy(img_path,new_path)
#             else:
#                 new_path = "C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-val\\normal\\" + str(col["abnormality id"]) + "-" + col["image file path"].split("\\")[-1] + "-" + img_name
#                 #res.append(new_path)
#                 shutil.copy(img_path,new_path)
#                 # put it into normal folder


# def write_into_test_folder():
#     for row,col in test_df.iterrows():
#         #res.append(os.listdir(col["image file path"]))
#         for img_name in os.listdir(col["image file path"]):
#             img_path = col["image file path"] + "\\" + img_name
#             if col["pathology"] == "MALIGNANT":
#                 # put this file to the malignant folder
#                 new_path = "C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-test\\malignant\\" + str(col["abnormality id"]) + "-" + col["image file path"].split("\\")[-1] + "-" + img_name
#                 #res.append(new_path)
#                 shutil.copy(img_path,new_path)
#             else:
#                 new_path = "C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-test\\normal\\" + str(col["abnormality id"]) + "-" + col["image file path"].split("\\")[-1] + "-" + img_name
#                 #res.append(new_path)
#                 shutil.copy(img_path,new_path)
                

# write_into_train_folder()
# write_into_val_folder()
# write_into_test_folder()


In [17]:
print(len(train))
print(len(test_df))
print(len(val))
print(len(os.listdir("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-train\\malignant")) + len(os.listdir("C:\\Users\\ae423\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-train\\normal")))
print(len(os.listdir("C:\\Users\\ae423\\OneDrive - University of Sussex\Desktop\\AlphanElmasDissertation\\cbis-val\\malignant")) + len(os.listdir("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-val\\normal")))
print(len(os.listdir("C:\\Users\\ae423\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-test\\malignant")) + len(os.listdir("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-test\\normal")))

2160
704
704
2160
704
704


In [ ]:
resnet50 = torchvision.models.resnet50(pretrained = True)

for param in resnet50.parameters():
    param.requires_grad = False

for param in list(resnet50.parameters())[-4:]:
    param.requires_grad = True




In [ ]:
import wandb
run = wandb.init(
    # Set the project where this run will be logged
    project="Pretrained Output 1",
    # Track hyperparameters and run metadata
)
wandb.login()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: alphanhilal (alphanhilal-university-of-sussex). Use `wandb login --relogin` to force relogin


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
class CLAHE():
    
    def __init__(self, clipLimit=2.0, tileGridSize=(16,16)):
        super().__init__
        self.clipLimit = clipLimit
        self.tileGridSize = tileGridSize

    def __call__(self,sample):
      img = np.array(sample)
      img = np.moveaxis(img,0,2)
      imgray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
      imgray = imgray.astype(np.uint16)
      clahe = cv2.createCLAHE(clipLimit=self.clipLimit, tileGridSize=self.tileGridSize)
      new_img = clahe.apply(imgray)
      new_img = cv2.cvtColor(new_img, cv2.COLOR_GRAY2BGR)
      new_img = cv2.cvtColor(new_img, cv2.COLOR_BGR2RGB)
      new_img = new_img.astype(np.float32)
      new_img = np.moveaxis(new_img,2,0)
      tensor = torch.tensor(new_img)
      #print(img.shape)
      return tensor


class GaussianNoise():

    def __init__(self,mean=0,stddev=0.01):
        super().__init__
        self.mean = mean
        self.stddev = stddev

    def __call__(self,sample):
        sample = sample.cpu()
        input_array = sample.data.numpy()

        noise = np.random.normal(loc=self.mean, scale=self.stddev, size=np.shape(input_array))

        out = np.add(input_array, noise)

        output_tensor = torch.from_numpy(out)
        out_tensor = Variable(output_tensor)
        out = out_tensor.cuda()
        out = out.float()
        return out

In [ ]:
import copy

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_model = None
        self.best_loss = None
        self.counter = 0
        self.status = ""

    def __call__(self, model, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
        elif self.best_loss - val_loss >= self.min_delta:
            self.best_model = copy.deepcopy(model.state_dict())
            self.best_loss = val_loss
            self.counter = 0
            self.status = f"Improvement found, counter reset to {self.counter}"
        else:
            self.counter += 1
            self.status = f"No improvement in the last {self.counter} epochs"
            if self.counter >= self.patience:
                self.status = f"Early stopping triggered after {self.counter} epochs."
                if self.restore_best_weights:
                    model.load_state_dict(self.best_model)
                return True
        return False

In [ ]:

def build_dataset(batch_size):
   
    transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(p=0.5), # data augmentation
    transforms.RandomRotation(degrees=(0,180)),
    transforms.RandomAffine(degrees = 0, translate = (0.2, 0.2)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), # normalization
    CLAHE(),
    GaussianNoise()

])
    indices = list(range(192))

    train_dir = 'C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-train'
    train_dataset = datasets.ImageFolder(train_dir, transforms_train)
    train_subset = torch.utils.data.Subset(train_dataset, indices)
    train_dataloader = torch.utils.data.DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)

    test_dir = 'C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-test'
    test_dataset = datasets.ImageFolder(test_dir, transforms_train)
    test_subset = torch.utils.data.Subset(test_dataset, indices)
    test_dataloader = torch.utils.data.DataLoader(test_subset, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)

    val_dir = 'C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-val'
    val_dataset = datasets.ImageFolder(val_dir, transforms_train)
    val_subset = torch.utils.data.Subset(val_dataset, indices)
    val_dataloader = torch.utils.data.DataLoader(val_subset, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)

    return train_dataloader,test_dataloader,val_dataloader


def build_resnet():
    model_ft = models.resnet50(weights='IMAGENET1K_V1')
    for param in model_ft.parameters():
        param.requires_grad = False
    num_ftrs = model_ft.fc.in_features # 2048
    model_ft.fc = nn.Linear(num_ftrs, 16) # 16 values will be come for the merged model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    return model_ft.to(device)

      

def build_optimizer(network, optimizer, learning_rate):

    
    optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer 


In [ ]:
import pandas as pd


numerical_predictions = []
def pathology_encoder(val):
    return "MALIGNANT" if val == "MALIGNANT" else "NORMAL"

calc_df = pd.read_csv("csv\calc_case_description_train_set.csv")
c_test = pd.read_csv("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\calc_case_description_test_set.csv")
calc_df["pathology"] = calc_df["pathology"].map(pathology_encoder)
cols_to_be_used = ["breast density","left or right breast","image view","abnormality id","calc type","calc distribution","assessment","pathology"]
new_calc_df = calc_df[cols_to_be_used]

mass_df = pd.read_csv("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\mass_case_description_train_set.csv")
m_test = pd.read_csv("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\mass_case_description_test_set.csv")
mass_df["pathology"] = mass_df["pathology"].map(pathology_encoder)
cols_to_be_used = ["breast_density","left or right breast","image view","abnormality id","mass shape","mass margins","assessment","pathology"]
new_mass_df = mass_df[cols_to_be_used]
new_mass_df.rename(columns={"breast_density":"breast density"},inplace=True)

bc_df = pd.concat([new_calc_df,new_mass_df])
lbl_encoder = LabelEncoder()
bc_df['left or right breast'] = lbl_encoder.fit_transform(bc_df['left or right breast'])
bc_df['image view'] = lbl_encoder.fit_transform(bc_df['image view'])
bc_df['calc type'] = lbl_encoder.fit_transform(bc_df['calc type'])
bc_df["calc distribution"] = lbl_encoder.fit_transform(bc_df['left or right breast'])
bc_df['pathology'] = lbl_encoder.fit_transform(bc_df['pathology'])
bc_df['mass shape'] = lbl_encoder.fit_transform(bc_df['mass shape'])
bc_df['mass margins'] = lbl_encoder.fit_transform(bc_df['mass margins'])

bc_df["breast density"] = bc_df["breast density"].astype(float)
bc_df["abnormality id"] = bc_df["abnormality id"].astype(float)
bc_df["assessment"] = bc_df["assessment"].astype(float)
bc_df["breast density"].fillna(np.mean(bc_df["breast density"]),inplace=True)
cols = ["breast density",'left or right breast',"image view","abnormality id","assessment", 'calc type','calc distribution', 'mass shape', 'mass margins']
scaler = StandardScaler()
bc_df[cols] = scaler.fit_transform(bc_df[cols])

x_train,y_train = bc_df[cols].iloc[:2160], bc_df["pathology"].iloc[:2160]
x_val,y_val = bc_df[cols].iloc[2160:], bc_df["pathology"].iloc[2160:]
df = pd.concat([c_test,m_test])


cols_to_be_used = ["breast_density","left or right breast","image view","abnormality id","mass shape","mass margins","assessment","pathology"]
new_mass_df = mass_df[cols_to_be_used]
new_mass_df.rename(columns={"breast_density":"breast density"},inplace=True)


df['left or right breast'] = lbl_encoder.fit_transform(df['left or right breast'])
df['image view'] = lbl_encoder.fit_transform(df['image view'])
df['calc type'] = lbl_encoder.fit_transform(df['calc type'])
df["calc distribution"] = lbl_encoder.fit_transform(df['left or right breast'])
df['pathology'] = lbl_encoder.fit_transform(df['pathology'])
df['mass shape'] = lbl_encoder.fit_transform(df['mass shape'])
df['mass margins'] = lbl_encoder.fit_transform(df['mass margins'])

df["breast density"] = df["breast density"].astype(float)
df["abnormality id"] = df["abnormality id"].astype(float)
df["assessment"] = df["assessment"].astype(float)


scaler = StandardScaler()
df[cols] = scaler.fit_transform(df[cols])
df["breast density"].fillna(np.mean(df["breast density"]),inplace=True)

df = df[["breast density","left or right breast","image view","abnormality id","assessment","calc type","calc distribution","pathology","mass shape","mass margins"]] 

x_test,y_test = df[cols], df["pathology"]

class NumericalClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(9, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16), # bu output degerini direkt olarak ensembe modele yollayacagiz cunku
            nn.ReLU()
        )

    def forward(self, x):
        output = self.linear_relu_stack(x)
        return output

y_train = np.reshape(y_train,(y_train.shape[0],1))
y_val = np.reshape(y_val,(y_val.shape[0],1))
y_test = np.reshape(y_test,(y_test.shape[0],1))


train_set = np.concatenate((x_train,y_train),axis=1)
val_set = np.concatenate((x_val,y_val),axis=1)
test_set = np.concatenate((x_test,y_test),axis=1)
train_set = torch.tensor(train_set, dtype=torch.float32)
val_set = torch.tensor(val_set, dtype=torch.float32)
test_set = torch.tensor(test_set, dtype=torch.float32)

# print(train_set.shape)
# print(val_set.shape)
# print(test_set.shape)

class EnsembleModel(nn.Module): # This neural network merges the 

    def __init__(self,resnet_input,numerical_input):
        super(EnsembleModel,self).__init__()
        self.resnet_input = resnet_input
        self.numerical_input = numerical_input
        self.total_params = torch.cat((self.resnet_input,self.numerical_input),1) 
        self.classifier = nn.Linear(32,1)


    def forward(self,param):
        param =  self.total_params
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        param = param.to(device)
        return F.sigmoid(self.classifier(param))   

In [ ]:
resnet_predictions = [] # The outputs of predictions will be stored in this array
numerical_predictions = [] # Numerical predictions will be stored in this array

In [ ]:
from sched import scheduler

def numerical_validation(network,val_dataloader,voutputs,vlabels,batch_size,step=0,epoch_num = 0, mode="validation"):
    model = NumericalClassifier()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()
    val_loader  = DataLoader(val_set,  batch_size=64, shuffle=False)
    with torch.no_grad():
        for step_num,values in enumerate(val_loader):
            if step_num != step:
                continue
            inputs,labels = values[:,:9],values[:,9] 
            inputs,labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            train_ensemble_model(voutputs,outputs,vlabels,batch_size,epoch_num,"validation")
            print("validation Numerical classifierdayim",step_num)
            validation(network,val_dataloader,batch_size,step_num+1,epoch_num= epoch_num, mode="validation")



def validation(network,val_dataloader,batch_size,step=0,epoch_num = 0, mode="validation"):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    network.eval()
    network.to(device)
    with torch.no_grad():
        for i, vdata in enumerate(val_dataloader):
            if i != step:
                continue
            print("Validation resnetteyim",i)
            vinputs, vlabels = vdata
            vinputs, vlabels = vinputs.to(device), vlabels.to(device)
            
            voutputs = torch.squeeze(network(vinputs))
            new_vlabels = torch.tensor(vlabels, dtype=torch.float32).to(device)
            numerical_validation(network,val_dataloader,voutputs,new_vlabels,batch_size,i,epoch_num,"validation")

def train_numerical_classifier(resnet_input,network,loader,optimizer,batch_size,target,step=0, epoch_num = 0, mode = "training"):
    batch_size = 64
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    optimizer = build_optimizer(NumericalClassifier(),"adam",learning_rate=0.001)
    model = NumericalClassifier()
    model.to(device)
    for batch_num, temp in enumerate(train_loader):
        inputs,labels = temp[:,:9], temp[:,9]
        inputs,labels = inputs.to(device),labels.to(device)
        if batch_num != step:
            continue
        optimizer.zero_grad()      
        numerical_output = model(inputs)
        optimizer.step()
        train_ensemble_model(resnet_input,numerical_output,target,batch_size, epoch_num, mode)
        print("Numerical classifierdayim",batch_num)
        if batch_num == 32:
            break
        train_epoch(network,loader,optimizer,batch_size,step=batch_num+1)
        # Train ensemble model loss u geriye dondurdukten sonra bir sonraki batch e gecmeden tekrardan resnet train kismina gecilmesi lazim ki orada bir sonraki batche gecilebilsin. 


loss_fn = nn.BCELoss()
def train_epoch(network, loader, optimizer, batch_size,step = 0, epoch_num = 0, mode = "training"):
    cumu_loss = 0
    network = build_resnet()    
    for batch_number, (data, target) in enumerate(loader): # batch batch butun datayi gezip loss degerlerini hesapliyorum
        # ensembe isini bitirdikten sonra bir sonraki batche gecilmesi gerekiyor bastan degil yani
        
        if batch_number != step:
            continue
        print("Trainingde batch sayim",batch_number)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        data,target = data.to(device),target.to(device)
        optimizer.zero_grad()
        network.to(device)
        # ➡ Forward pass
        prediction = network(data)
        prediction = torch.squeeze(prediction)
        new_target = torch.tensor(target, dtype=torch.float32)
        optimizer.step()
        train_numerical_classifier(prediction,network,loader,optimizer,batch_size,new_target,batch_number, epoch_num, mode) # batch number baslangicta 0 burda
        if batch_number == 32:
            break


batch_size = 0
def train(config=None):
    global batch_size
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        train_dataloader,test_dataloader,val_dataloader = build_dataset(config.batch_size)
        network = build_resnet()
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)
        #scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.5, total_iters=10)
        es = EarlyStopping()
        best_val_loss = np.inf
        PATH = './cifar_net.pth'
        batch_size = config.batch_size

        for epoch in range(config.epochs):
            network.train(True)
            print("Epoch num is", epoch)
            train_epoch(network, train_dataloader, optimizer,batch_size,step=0,epoch_num=epoch,mode="training")
            print("Validationa gectim epoch sayim",epoch)
            validation(network,val_dataloader,batch_size,step=0,epoch_num=epoch, mode="validation")


resnet_predictions = []


optimizer = build_optimizer(NumericalClassifier(),"adam",learning_rate=0.001)
current_mode = "training"
predictions = []
targets = []
epoch_val = 0   
loss_values = []
val_loss_values = []
prediction_list = []
target_list = []
metrics = []
val_metrics = []
best_loss = np.inf

def train_ensemble_model(resnet_input,numerical_input,target,batch_size, epoch_num=0, mode="training"):
    global epoch_val
    global current_mode
    global metrics
    global val_metrics
    global best_loss
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    ensemble_model = EnsembleModel(resnet_input,numerical_input)
    ensemble_model.to(device)
    if mode == "train":
        ensemble_model.train()
    else: # in validation mode
        ensemble_model.eval()
    #print("Parametreler",ensemble_model.total_params.shape)
    i = 0
    if current_mode != mode: # this condition states that if the epoch has finished

        #y_true_tensor = torch.tensor(targets)
        #y_pred_tensor = torch.tensor(predictions)
        #print(y_pred_tensor)
        #print(y_true_tensor)
        #sys.exit()
        if current_mode == "training":
            # print(predictions)
            # print(targets)
            # sys.exit()
            avg_loss = sum(loss_values) / len(loss_values)
            metrics = np.array(metrics)
            wandb.log({"training_loss": avg_loss, "epoch": epoch_val+1})
            wandb.log({"training_accuracy": np.mean(metrics[:,0]), "epoch": epoch_val+1})
            wandb.log({"training_precision": np.mean(metrics[:,1]), "epoch": epoch_val+1})
            wandb.log({"training_recall": np.mean(metrics[:,2]), "epoch": epoch_val+1})
            wandb.log({"training_f1": np.mean(metrics[:,3]), "epoch": epoch_val+1})
      
            loss_values.clear()
            predictions.clear()
            targets.clear()
            prediction_list.clear()
            target_list.clear()
            metrics = metrics.tolist()
            metrics.clear()

        if current_mode == "validation":
            #print("Kod buraya girebiliyor")
            val_metrics = np.array(val_metrics)
            avg_val_loss = sum(val_loss_values) / len(val_loss_values)
            if avg_val_loss < best_loss:
                best_loss = avg_val_loss
                torch.save(ensemble_model.state_dict(),"best_model_params.pt")
            wandb.log({"validation_loss": avg_val_loss, "epoch": epoch_val+1})
            wandb.log({"validation_accuracy": np.mean(val_metrics[:,0]), "epoch": epoch_val+1})
            wandb.log({"validation_precision": np.mean(val_metrics[:,1]), "epoch": epoch_val+1})
            wandb.log({"validation_recall": np.mean(val_metrics[:,2]), "epoch": epoch_val+1})
            wandb.log({"validation_f1_score": np.mean(val_metrics[:,3]), "epoch": epoch_val+1})
            val_loss_values.clear()
            predictions.clear()
            targets.clear()
            prediction_list.clear()
            target_list.clear()
            val_metrics = val_metrics.tolist()
            val_metrics.clear()

        current_mode = mode


    
   
    prediction_prob = ensemble_model(ensemble_model.total_params) # Buradan sigmoid cikisi 0 ile 1 arasinda bir deger donecek
    prediction_label = prediction_prob.round() # sigmoidden gelen verileri burada 0 ya da 1 e yuvarliyoruz thresholdu 0.5 kabul ederek
    target = torch.reshape(target, (target.shape[0],1))
    
    targets.extend(target.cpu().numpy().astype(int))
    for t in targets:
        target_list.append(t.item())
    loss_val = nn.BCELoss()(prediction_label,target) / batch_size
    # print("Loss val is", loss_val)
    if mode == "training":
        print("training batch")
        loss_values.append(loss_val.item())
        predictions.extend(prediction_label.detach().cpu().numpy().astype(int))
        for i in predictions:
            prediction_list.append(i.item())

        batch_acc = accuracy_score(prediction_list,target_list)
        batch_preicision = precision_score(prediction_list,target_list)
        batch_recall = recall_score(prediction_list,target_list)
        batch_f1 = f1_score(prediction_list,target_list)


        wandb.log({"batch_loss": loss_val.item(), "epoch": epoch_val+1})
        wandb.log({"batch_accuracy":batch_acc , "epoch": epoch_val+1})
        wandb.log({"batch_precision": batch_preicision, "epoch": epoch_val+1})
        wandb.log({"batch_recall": batch_recall, "epoch": epoch_val+1})
        wandb.log({"batch_f1": batch_f1, "epoch": epoch_val+1})
        metrics.append([batch_acc,batch_preicision,batch_recall,batch_f1])
        loss_val.backward(retain_graph=True)
        optimizer.step() 
        optimizer.zero_grad()

        predictions.clear()
        targets.clear()
        prediction_list.clear()
        target_list.clear()
    else: # mode == "validation"
        val_loss_values.append(loss_val.item())
        #print(val_loss_values)
        predictions.extend(prediction_label.cpu().numpy())
        for i in predictions:
            prediction_list.append(i.item())
        batch_val_acc = accuracy_score(prediction_list,target_list)
        batch_val_preicision = precision_score(prediction_list,target_list)
        batch_val_recall = recall_score(prediction_list,target_list)
        batch_val_f1 = f1_score(prediction_list,target_list)
        wandb.log({"val_batch_loss": loss_val.item(), "epoch": epoch_val+1})
        wandb.log({"val_batch_accuracy": batch_val_acc, "epoch": epoch_val+1})
        wandb.log({"val_precision": batch_val_preicision, "epoch": epoch_val+1})
        wandb.log({"val_batch_recall": batch_val_recall, "epoch": epoch_val+1})
        wandb.log({"val_batch_f1": batch_val_f1, "epoch": epoch_val+1})
        val_metrics.append([batch_val_acc,batch_val_preicision,batch_val_recall,batch_val_f1])
        # bu islemden sonra batch bitti ve cikmasi lazim


In [ ]:
def test():
    network = build_resnet()
    best_checkpoint = torch.load('./best_model_params.pth')

    network.eval()
    y_true = []
    y_pred = []

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    train_dataloader,test_dataloader,val_dataloader = build_dataset(64)
    for epoch in range(5):
        with torch.no_grad():
            for images, labels in test_dataloader:
                images = images.to(device)
                outputs = network(images)
                pred_prob = torch.sigmoid(outputs)
                predicted = pred_prob.round()
                #_, predicted = torch.max(outputs, 1)
                resnet_predictions.extend(outputs.cpu().numpy())
                y_pred.extend(predicted.cpu().numpy()) # bu benim ust mlp ye koyacagim input oluyor su an
                y_true.extend(labels.cpu().numpy())
            
            
            test_acc = accuracy_score(y_pred,y_true)
            test_preicision = precision_score(y_pred,y_true)
            test_recall = recall_score(y_pred,y_true)
            test_f1 = f1_score(y_pred,y_true)


            print(f'Accuracy: {test_acc}')
            print(f'Precision: {test_preicision}')
            print(f'Recall: {test_recall}')
            print(f'F1 Score: {test_f1}')
            
            wandb.log({"test precision":test_preicision})
            wandb.log({"test recall":test_recall})
            wandb.log({"test f1 score":test_f1})
            wandb.log({"test accuracy":test_acc})

In [ ]:
sweep_config = {
    'method': 'grid'
    }

metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'optimizer': {
        'values': ['adam']
        },
    'learning_rate': {
        "values" : [0.0001] # 0.0001 vermeyi bi dene bakalim
      },
      'batch_size': {
        "values" : [64]
      },
      "epochs" : {
        "values" : [6]
      }
    }
 
sweep_config['parameters'] = parameters_dict


import pprint
pprint.pprint(sweep_config)
sweep_id = wandb.sweep(sweep_config, project="Pretrained Output 1")

wandb.agent(sweep_id, train)



{'method': 'grid',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'values': [64]},
                'epochs': {'values': [6]},
                'learning_rate': {'values': [0.0001]},
                'optimizer': {'values': ['adam']}}}


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: nwnrq8x9
Sweep URL: https://wandb.ai/alphanhilal-university-of-sussex/Pretrained%20Output%201/sweeps/nwnrq8x9


wandb: Sweep Agent: Waiting for job.
wandb: Job received.


: 